In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np


In [ ]:
# Load in data
data = torch.load("D:/GitHub/solar-forecasting/data/data_pipeline.pt")
X_tensor = data['X_tensor']


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, bidirectional=True, dropout_p=0.3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            dropout=dropout_p, bidirectional=bidirectional,
                            batch_first=True)

        self.dropout = nn.Dropout(dropout_p)
        direction_factor = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_size * direction_factor, 1)  # Regression output

    def forward(self, x):
        out, _ = self.lstm(x)
        last_time_step_out = out[:, -1, :]
        dropped = self.dropout(last_time_step_out)  # MC Dropout always applied
        out = self.fc(dropped)
        return out.squeeze()

In [ ]:
def enable_mc_dropout(model):
    for m in model.modules():
        if isinstance(m, nn.Dropout):
            m.train()  # Force dropout ON

In [ ]:
def mc_predict(model, x_batch, n_passes=50):
    model.eval()
    enable_mc_dropout(model)

    preds = []
    for _ in range(n_passes):
        with torch.no_grad():
            preds.append(model(x_batch).cpu().numpy())

    preds = np.stack(preds, axis=0)  # shape: (n_passes, batch_size)
    mean = preds.mean(axis=0)        # shape: (batch_size,)
    std = preds.std(axis=0)          # shape: (batch_size,)
    return mean, std

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

# Custom Dataset class for real-time evaluation (no labels or sample weights)
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, X):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

# Create the Dataset with just the input data (X) for real-time evaluation
real_time_dataset = SequenceDataset(X_tensor)

# Create the DataLoader for real-time evaluation
real_time_loader = DataLoader(real_time_dataset, batch_size=32, shuffle=False)

# Example: Using the DataLoader for real-time evaluation
for batch in real_time_loader:
    # Here you can use your model for prediction
    # For example:
    # predictions = model(batch)
    print(batch.shape)  # Check the batch shape


In [ ]:
input_size = 21  # <-- replace with your real input size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMModel(input_size=input_size)
model.load_state_dict(torch.load("D:/GitHub/solar-forecasting/models/best_model_v2.pt", map_location=device))
model.to(device)
model.eval()


In [ ]:
all_means = []
all_stds = []

model.eval()
enable_mc_dropout(model)

with torch.no_grad():
    for X_batch in real_time_loader:
        X_batch = X_batch.to(device)
        batch_mean, batch_std = mc_predict(model, X_batch, n_passes=50)
        all_means.extend(batch_mean)
        all_stds.extend(batch_std)

In [ ]:
all_means = np.array(all_means)
all_stds = np.array(all_stds)
mean_linear = all_means
std_linear = ((all_means + all_stds)) - mean_linear  # approximate upper range

for i in range(len(mean_linear)):
    print(f"Sample {i}: Flare = {mean_linear[i]:.2f}, ± {std_linear[i]:.2f}")